In [20]:
!pip install -Uq dataidea


In [1]:
from sklearn.preprocessing import StandardScaler

In [2]:
import dataidea as di

In [3]:
dataset = di.load_dataset('boston')

In [4]:
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
X = dataset.drop('MEDV', axis=1)
y = dataset['MEDV']

In [6]:

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [17]:
from torch import tensor,float32
from torch.utils.data import TensorDataset,DataLoader
from torch.nn import MSELoss,Sequential,ReLU,Linear
from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [10]:
X = tensor(X, dtype=float32)
y = tensor(y, dtype=float32).view(-1,1)

<ipython-input-10-267da82c3588>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = tensor(X, dtype=float32)
<ipython-input-10-267da82c3588>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = tensor(y, dtype=float32).view(-1,1)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [19]:
model = Sequential(Linear(13,80),
                   ReLU(),
                   Linear(80,50),
                   ReLU(),
                   Linear(50,30),ReLU(),
                   Linear(30,10),ReLU(),
                   Linear(10,1))

In [20]:
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.01)

In [21]:
epochs = 200
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch: {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')

Epoch: 1/200, Loss: 237.55720057854285
Epoch: 2/200, Loss: 40.481336483588585
Epoch: 3/200, Loss: 21.659417170744675
Epoch: 4/200, Loss: 15.7602164378533
Epoch: 5/200, Loss: 14.750603125645565
Epoch: 6/200, Loss: 12.966458485676693
Epoch: 7/200, Loss: 12.475861246769245
Epoch: 8/200, Loss: 13.246174408839298
Epoch: 9/200, Loss: 14.067360437833345
Epoch: 10/200, Loss: 12.662370214095482
Epoch: 11/200, Loss: 11.71904419018672
Epoch: 12/200, Loss: 10.41060116657844
Epoch: 13/200, Loss: 10.030337636287395
Epoch: 14/200, Loss: 9.34498446721297
Epoch: 15/200, Loss: 8.497246091182415
Epoch: 16/200, Loss: 12.150963948323177
Epoch: 17/200, Loss: 11.09373321900001
Epoch: 18/200, Loss: 8.28243138239934
Epoch: 19/200, Loss: 9.3781787615556
Epoch: 20/200, Loss: 9.370344913922823
Epoch: 21/200, Loss: 7.385843937213604
Epoch: 22/200, Loss: 8.115639649904692
Epoch: 23/200, Loss: 10.414042862562033
Epoch: 24/200, Loss: 8.773859684283916
Epoch: 25/200, Loss: 7.808255580755381
Epoch: 26/200, Loss: 6.6167

In [22]:
import pandas as pd
import plotly.express as px
from torch import no_grad
model.eval()
with no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.numpy().flatten()
    y_test = y_test.numpy().flatten()
pred = pd.DataFrame(data = {'Actual':y_test,
                            'Predict':y_pred})
fig = px.scatter(pred, x='Actual', y='Predict',title='a vs p',trendline='ols')
fig.show()
